# `megamerge`

The aim is to create a merge index / overlap list as fast as possible.

This may involve `numpy`/`scipy`, or we may delve into Rust.

The aim is to complete the part of the merge task that takes exponential time, such that python can perform the aggregations which take linear time.

It looks like the longest road segment is on H006 its 1050km long (or 3200km if you ignore POES). ANy road segment longer than this is probably not practically useful.

In [1]:
import pandas as pd
import numpy as np

In [24]:
segments = pd.DataFrame(
    columns=["road", "cwy", "slk_from", "slk_to"],
    data=[
        ["H001", "L",   0, 100],
        ["H001", "L", 100, 200],
        ["H001", "L", 200, 300],
        ["H001", "L", 300, 400],

        ["H001", "R",   0, 100],
    ]
)

data = pd.DataFrame(
    columns=["road", "cwy", "slk_from", "slk_to", "measure", "category"],
    data=[                             # overlaps lengths
        ["H001", "L", 50,  140,  1.0, "A"],  # 50  40   0  0    [50, 40,  0,  0]
        ["H001", "L", 140, 160,  2.0, "B"],  #  0  20   0  0    [ 0, 20,  0,  0]
        ["H001", "L", 160, 180,  3.0, "B"],  #  0  20   0  0    [ 0, 20,  0,  0]
        ["H001", "L", 180, 220,  4.0, "B"],  #  0  20  20  0    [ 0, 20, 20,  0]
        ["H001", "L", 220, 240,  5.0, "C"],  #  0   0  20  0    [ 0,  0, 20,  0]
        ["H001", "L", 240, 260,  5.0, "C"],  #  0   0  20  0    [ 0,  0, 20,  0]
        ["H001", "L", 260, 280,  6.0, "D"],  #  0   0  20  0    [ 0,  0, 20,  0]
        ["H001", "L", 280, 300,  7.0, "E"],  #  0   0  20  0    [ 0,  0, 20,  0]
        ["H001", "L", 300, 320,  8.0, "F"],  #  0   0     20    [ 0,  0,  0, 20]

        ["H001", "R",  10,  80,  9.0, "G"],  #  0   0     20
        ["H001", "R",  80, 120, 10.0, "H"],  #  0   0     20
    ]
)

















expected_result = pd.DataFrame(
    columns=["road", "cwy", "slk_from", "slk_to",  "measure longest value",  "category longest value"],
    data=[
        ["H001", "L",   0, 100,  1.0,  "A"],
        ["H001", "L", 100, 200,  1.0,  "B"],
        ["H001", "L", 200, 300,  5.0,  "C"],
        ["H001", "L", 300, 400,  8.0,  "F"],

        ["H001", "R",   0, 100,  9.0,  "G"],
    ]
)


## Old Merge Method

Using `dtimsprep` library which accuratly reproduces the results from the old merge macro.

In [25]:
import dtimsprep.merge as merge
result = merge.on_slk_intervals(
    segments,
    data,
    ["road", "cwy"],
    [
        merge.Action('measure',  rename="measure longest value",  aggregation=merge.Aggregation.KeepLongest()),
        merge.Action('category', rename="category longest value", aggregation=merge.Aggregation.KeepLongest()),
    ]
)
pd.testing.assert_frame_equal(result, expected_result)

## MegaMerge

Now lets try do this FASTER

In [6]:
from typing import Union

# preprocess the index on target segmentation to reduce the number of columns we need to work with
def get_categorical_index(segmentation:pd.DataFrame, categories:list[str]) -> Union[pd.Index, pd.MultiIndex]:
    """
    categories should not be in the index
    """
    return (
        segmentation
        .loc[:, list(categories)]
        .drop_duplicates()
        .set_index(categories)
        .index
    )

In [27]:
# Squish all the categorical indecies
seg_idx = get_categorical_index(segments, ["road", "cwy"])
seg_idx

MultiIndex([('H001', 'L'),
            ('H001', 'R')],
           names=['road', 'cwy'])

In [28]:
# modify the segmentation to use the squished index
segments_with_compacted_index = segments.copy()
segments_with_compacted_index = segments_with_compacted_index.drop(columns=["road","cwy"])
segments_with_compacted_index["compacted_category_index"] = seg_idx.get_indexer(segments.loc[:, ["road", "cwy"]])
segments_with_compacted_index=segments_with_compacted_index.set_index("compacted_category_index")
segments_with_compacted_index

,slk_from,slk_to
compacted_category_index,,
0,0,100
0,100,200
0,200,300
0,300,400
1,0,100


In [29]:
# modify the data to use the squished index
data_with_compacted_index = data.copy()
data_with_compacted_index = data_with_compacted_index.drop(columns=["road","cwy"])
data_with_compacted_index["compacted_category_index"] = seg_idx.get_indexer(data.loc[:, ["road", "cwy"]])
data_with_compacted_index=data_with_compacted_index.set_index("compacted_category_index")
data_with_compacted_index

,slk_from,slk_to,measure,category
compacted_category_index,,,,
0,50,140,1.0,A
0,140,160,2.0,B
0,160,180,3.0,B
0,180,220,4.0,B
0,220,240,5.0,C
0,240,260,5.0,C
0,260,280,6.0,D
0,280,300,7.0,E
0,300,320,8.0,F


In [30]:
# now lets look at the matching chunk from both dataframes
D = data_with_compacted_index.loc[0,:]
S = segments_with_compacted_index.loc[0,:]
display(D)
display(S)


,slk_from,slk_to,measure,category
compacted_category_index,,,,
0,50,140,1.0,A
0,140,160,2.0,B
0,160,180,3.0,B
0,180,220,4.0,B
0,220,240,5.0,C
0,240,260,5.0,C
0,260,280,6.0,D
0,280,300,7.0,E
0,300,320,8.0,F


,slk_from,slk_to
compacted_category_index,,
0,0,100
0,100,200
0,200,300
0,300,400


In [45]:
overlap_minimum = np.maximum(
    D["slk_from"].values,
    np.expand_dims(S["slk_from"].values,1),
)
overlap_minimum

array([[ 50, 140, 160, 180, 220, 240, 260, 280, 300],
       [100, 140, 160, 180, 220, 240, 260, 280, 300],
       [200, 200, 200, 200, 220, 240, 260, 280, 300],
       [300, 300, 300, 300, 300, 300, 300, 300, 300]], dtype=int64)

In [43]:
overlap_maximum = np.minimum(
    D["slk_to"].values,
    np.expand_dims(S["slk_to"].values,1),
)
np.transpose(overlap_maximum)

array([[100, 140, 140, 140],
       [100, 160, 160, 160],
       [100, 180, 180, 180],
       [100, 200, 220, 220],
       [100, 200, 240, 240],
       [100, 200, 260, 260],
       [100, 200, 280, 280],
       [100, 200, 300, 300],
       [100, 200, 300, 320]], dtype=int64)

In [44]:
overlap = np.maximum(0, overlap_maximum - overlap_minimum)
np.transpose(overlap)
#overlap = overlap.astype("f8")
#overlap[overlap==-100]=np.nan
#overlap

array([[50, 40,  0,  0],
       [ 0, 20,  0,  0],
       [ 0, 20,  0,  0],
       [ 0, 20, 20,  0],
       [ 0,  0, 20,  0],
       [ 0,  0, 20,  0],
       [ 0,  0, 20,  0],
       [ 0,  0, 20,  0],
       [ 0,  0,  0, 20]], dtype=int64)

In [14]:

D.reset_index().join(pd.DataFrame(np.transpose(overlap), columns=pd.MultiIndex.from_arrays(S.loc[:,["slk_from",	"slk_to"]].values.transpose())))

C:\Users\e81005\Miniconda39\lib\site-packages\pandas\core\frame.py:9126: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (1 levels on the left,2 on the right)
  return merge(


,compacted_category_index,slk_from,slk_to,measure,category,"(0, 100)","(100, 200)","(200, 300)","(300, 400)"
0,0,50,140,1.0,A,50,40,0,0
1,0,140,160,2.0,B,0,20,0,0
2,0,160,180,3.0,B,0,20,0,0
3,0,180,220,4.0,B,0,20,20,0
4,0,220,240,5.0,C,0,0,20,0
5,0,240,260,5.0,C,0,0,20,0
6,0,260,280,6.0,D,0,0,20,0
7,0,280,300,7.0,E,0,0,20,0
8,0,300,320,8.0,F,0,0,0,20


## Lets Go Big!

In [47]:
# generate segmentation
big_segments = pd.DataFrame(
    columns=["road", "cwy", "slk_from", "slk_to"],
    data   =[
        ["H001", "L",   index*10, index*10+10] for index in range(int(3000 / 0.100))
    ]
)
big_segments

,road,cwy,slk_from,slk_to
0,H001,L,0,10
1,H001,L,10,20
2,H001,L,20,30
3,H001,L,30,40
4,H001,L,40,50
...,...,...,...,...
29995,H001,L,299950,299960
29996,H001,L,299960,299970
29997,H001,L,299970,299980
29998,H001,L,299980,299990


In [48]:
big_data = pd.DataFrame(
    columns= ["road", "cwy", "slk_from", "slk_to", "measure"],
    data   = [
        ["H001", "L",   (index)*1, (index+1)*1, index] for index in range(int(3000 / 0.010))
    ]
)
big_data

,road,cwy,slk_from,slk_to,measure
0,H001,L,0,1,0
1,H001,L,1,2,1
2,H001,L,2,3,2
3,H001,L,3,4,3
4,H001,L,4,5,4
...,...,...,...,...,...
299995,H001,L,299995,299996,299995
299996,H001,L,299996,299997,299996
299997,H001,L,299997,299998,299997
299998,H001,L,299998,299999,299998


In [49]:
# Squish all the categorical indecies
seg_idx = get_categorical_index(big_segments, ["road", "cwy"])
print(seg_idx)

# modify the segmentation to use the squished index
segments_with_compacted_index = big_segments.copy()
segments_with_compacted_index = segments_with_compacted_index.drop(columns=["road","cwy"])
segments_with_compacted_index["compacted_category_index"] = seg_idx.get_indexer(big_segments.loc[:, ["road", "cwy"]])
segments_with_compacted_index=segments_with_compacted_index.set_index("compacted_category_index")
display(segments_with_compacted_index)

# modify the data to use the squished index
data_with_compacted_index = big_data.copy()
data_with_compacted_index = data_with_compacted_index.drop(columns=["road","cwy"])
data_with_compacted_index["compacted_category_index"] = seg_idx.get_indexer(big_data.loc[:, ["road", "cwy"]])
data_with_compacted_index=data_with_compacted_index.set_index("compacted_category_index")
display(data_with_compacted_index)

D = data_with_compacted_index.loc[0,:]
S = segments_with_compacted_index.loc[0,:]

MultiIndex([('H001', 'L')],
           names=['road', 'cwy'])


,slk_from,slk_to
compacted_category_index,,
0,0,10
0,10,20
0,20,30
0,30,40
0,40,50
...,...,...
0,299950,299960
0,299960,299970
0,299970,299980


,slk_from,slk_to,measure
compacted_category_index,,,
0,0,1,0
0,1,2,1
0,2,3,2
0,3,4,3
0,4,5,4
...,...,...,...
0,299995,299996,299995
0,299996,299997,299996
0,299997,299998,299997


In [98]:
def overlap(segmentation,data,offset,chunk_size=1000, proximity_threshold=0):
    overlap_minimum = np.maximum(
        segmentation["slk_from"].values[offset:offset+chunk_size],
        np.expand_dims(data["slk_from"].values,1),
    )
    overlap_maximum = np.minimum(
        segmentation["slk_to"].values[offset:offset+chunk_size],
        np.expand_dims(data["slk_to"].values,1),
    )
    return np.maximum(overlap_maximum - overlap_minimum - proximity_threshold, 0)

In [99]:
print(f"processing {len(D):,.0f} datapoints onto {len(S):,.0f} segments is")
print(f"equivalent to checking every pixle in 4k video at 24fps for {len(D)*len(S)*8 / ((1920*4)*(1080*4)*3)/24/60:.1f} minutes")
print(f"We expect it to take at about that long.")

processing 300,000 datapoints onto 30,000 segments is
equivalent to checking every pixle in 4k video at 24fps for 0.5 minutes
We expect it to take at about that long.


In [108]:
chunk_size = 200
num_chunks = int(len(S) / chunk_size)
print("."*num_chunks)
print("")
import time
start_time = time.perf_counter_ns()
last_backspace=0
for i in range(num_chunks):
    overlap(
        segmentation        = S,
        data                = D,
        offset              = i*chunk_size,
        chunk_size          = chunk_size,
        proximity_threshold = 0
    )
    cur_time = time.perf_counter_ns();
    elapsed = (cur_time-start_time)/1e9
    #print("\b" * last_backspace,end="\r")
    #to_print = f"remaining: {(num_chunks-i)*(elapsed/(i+1)):.0f} seconds      "
    last_backspace = len(to_print)
    print(f"elapsed: {elapsed:.0f} remaining: {(num_chunks-i)*(elapsed/(i+1)):.0f} seconds     ",end="\r")
print(f" " * last_backspace, end="\r")
end_time = time.perf_counter_ns();
print(f"elapsed {(end_time-start_time)/1e9:.0f} seconds")

......................................................................................................................................................

elapsed 110 seconds       seconds      


In [141]:
def overlap(segmentation,data,offset,chunk_size=1000, proximity_threshold=0):
    overlap_minimum = np.maximum(
        segmentation["slk_from"].values[offset:offset+chunk_size],
        np.expand_dims(data["slk_from"].values,1),
    )
    overlap_maximum = np.minimum(
        segmentation["slk_to"].values[offset:offset+chunk_size],
        np.expand_dims(data["slk_to"].values,1),
    )
    return np.maximum(overlap_maximum - overlap_minimum - proximity_threshold, 0)

def merg (segmentation, data, chunk_size, proximity_threshold):
    num_chunks = int(len(segmentation) / chunk_size);
    for offset in range(0, len(segmentation), chunk_size):
        ovlp = overlap(
            segmentation        = segmentation,
            data                = data,
            offset              = offset,
            chunk_size          = chunk_size,
            proximity_threshold = proximity_threshold
        )
        yield (
            offset,
            [
                np.array(
                    [
                        (ind := np.nonzero(item)[0]) + offset,
                        np.take(item, ind) + proximity_threshold
                    ]
                )
                for item in np.transpose(ovlp)
            ]
        )

In [142]:
for offset, chunk in merg(S.iloc[:100], D.iloc[:1000], chunk_size=300, proximity_threshold=0):
    print(chunk)

[array([[0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], dtype=int64), array([[10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
       [ 1,  1,  1,  1,  1,  1,  1,  1,  1,  1]], dtype=int64), array([[20, 21, 22, 23, 24, 25, 26, 27, 28, 29],
       [ 1,  1,  1,  1,  1,  1,  1,  1,  1,  1]], dtype=int64), array([[30, 31, 32, 33, 34, 35, 36, 37, 38, 39],
       [ 1,  1,  1,  1,  1,  1,  1,  1,  1,  1]], dtype=int64), array([[40, 41, 42, 43, 44, 45, 46, 47, 48, 49],
       [ 1,  1,  1,  1,  1,  1,  1,  1,  1,  1]], dtype=int64), array([[50, 51, 52, 53, 54, 55, 56, 57, 58, 59],
       [ 1,  1,  1,  1,  1,  1,  1,  1,  1,  1]], dtype=int64), array([[60, 61, 62, 63, 64, 65, 66, 67, 68, 69],
       [ 1,  1,  1,  1,  1,  1,  1,  1,  1,  1]], dtype=int64), array([[70, 71, 72, 73, 74, 75, 76, 77, 78, 79],
       [ 1,  1,  1,  1,  1,  1,  1,  1,  1,  1]], dtype=int64), array([[80, 81, 82, 83, 84, 85, 86, 87, 88, 89],
       [ 1,  1,  1,  1,  1,  1,  1,  1,  1,  1]], dtype=int64), ar

In [127]:
len(S["slk_from"].values[10:10+10])

10